In [6]:
#system imports 
import os
import json
from dotenv import load_dotenv
load_dotenv('/Users/joe.constantino/Desktop/playground/tableau_langchain/.env')

#base langchain imports
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
#from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser

#import tableau_langchain libraries
from hbiQuery import get_data
from readMetadata import read
from setPrompt import instantiate_prompt, get_values
from prompts.nlq_to_vds import prompt

In [2]:
#read in environment variables
datasource = os.getenv('DATASOURCE_LUID')
read_metadata_url = os.getenv('READ_METADATA')
query_datasource = os.getenv('QUERY_DATASOURCE')
token = os.getenv('AUTH_TOKEN')

In [3]:
from hbiQuery import get_data
from readMetadata import read
from prompts.nlq_to_vds import prompt
import json, requests, os
from setPrompt import instantiate_prompt, get_values

In [4]:
metadata = read(read_url=read_metadata_url, datasource_luid=datasource, auth_secret=token)
prompt = instantiate_prompt(metadata = metadata)

reading in your field metadata
looking up field values


In [11]:
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

prompt_template = ChatPromptTemplate.from_messages([
  SystemMessage(content=json.dumps(prompt)),
  ("user", "{utterance}")])

output_parser = StrOutputParser()

chain = prompt_template | llm | output_parser

output = chain.invoke("which products perform the best in the west region?")
print(output)

Reasoning:
To determine which products perform the best in the West region, we will query the total sales and profit for each product in that specific region. This will help identify the top-performing products based on sales and profit metrics.

JSON_payload:
```json
{
  "fields": [
    {
      "fieldCaption": "Product Name"
    },
    {
      "fieldCaption": "Sales",
      "function": "SUM",
      "maxDecimalPlaces": 2
    },
    {
      "fieldCaption": "Profit",
      "function": "SUM",
      "maxDecimalPlaces": 2
    }
  ],
  "filters": [
    {
      "fieldCaption": "Region",
      "filterType": "SET",
      "values": ["West"],
      "exclude": false
    }
  ]
}
```


In [15]:
from tabulate import tabulate
def showAnswer(chain_output):
    parsed_output = chain_output.split('JSON_payload')[1]
    match = re.search(r'{.*}', parsed_output, re.DOTALL)
    if match:
        json_string = match.group(0)
    payload = json.loads(json_string)
    return tabulate(get_data(query_url = query_datasource, datasource_luid = datasource, auth_secret = token, query=payload),
                    headers="keys",tablefmt="grid")

print(showAnswer(output))

+---------------------------------------------------------------------------------------------------------------+--------------+---------------+
| Product Name                                                                                                  |   SUM(Sales) |   SUM(Profit) |
+===============================================================================================================+==============+===============+
| DAX Metal Frame, Desktop, Stepped-Edge                                                                        |        20.24 |          7.89 |
+---------------------------------------------------------------------------------------------------------------+--------------+---------------+
| Hon Comfortask Task/Swivel Chairs                                                                             |       638.29 |        -31.91 |
+---------------------------------------------------------------------------------------------------------------+--------------+--